# Processing Non Diabetic data

In [2]:
# Classic Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from collections import Counter
from collections import defaultdict

#Advanced Visualization Libraries
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected = True) #enables plotly plots to be displayed in notebook
cmap1 = "gist_gray"

#Models
from lightgbm import LGBMClassifier
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

#Metrics, Preprocessing and Tuning Tools
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
import missingno as msno
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

#Customization
import warnings
warnings.filterwarnings("ignore")
from termcolor import colored

In [3]:
#Read the data
non_diabetic_df = pd.read_csv('NonDiabetic.csv')

In [4]:
non_diabetic_df.shape

(5957566, 14)

In [5]:
non_diabetic_df.head()

,Patient,Diagnosis_Code,DiagnosisGroup,Diagnosis_Description,ResultInterpretation,ResultValue,ResultValueUnits,TestItemCode_Description,ObservationValue,ObservationCode_Description,ObservationCode_ObservationValueUnits_Code,Age,Gender_Name,Citizenship_Description
0,171439,U07.1,U07,COVID 19 virus identified,Normal,Not Detected,NaN,COVID-19 PCR,20.0,Respiratory Rate,bpm,48,Female,EMIRATI
1,171439,U07.1,U07,COVID 19 virus identified,Normal,Not Detected,NaN,COVID-19 PCR,118.0,Systolic Blood Pressure,mmHg,48,Female,EMIRATI
2,171439,U07.1,U07,COVID 19 virus identified,Normal,Not Detected,NaN,COVID-19 PCR,66.0,Diastolic Blood Pressure,mmHg,48,Female,EMIRATI
3,171439,U07.1,U07,COVID 19 virus identified,Normal,Not Detected,NaN,COVID-19 PCR,56.0,Weight,kg,48,Female,EMIRATI
4,171439,U07.1,U07,COVID 19 virus identified,Normal,Not Detected,NaN,COVID-19 PCR,142.0,Height,cm,48,Female,EMIRATI


In [7]:
non_diabetic_df['Target'] = 0

In [8]:
non_diabetic_df.head()

,Patient,Diagnosis_Code,DiagnosisGroup,Diagnosis_Description,ResultInterpretation,ResultValue,ResultValueUnits,TestItemCode_Description,ObservationValue,ObservationCode_Description,ObservationCode_ObservationValueUnits_Code,Age,Gender_Name,Citizenship_Description,Target
0,171439,U07.1,U07,COVID 19 virus identified,Normal,Not Detected,NaN,COVID-19 PCR,20.0,Respiratory Rate,bpm,48,Female,EMIRATI,0
1,171439,U07.1,U07,COVID 19 virus identified,Normal,Not Detected,NaN,COVID-19 PCR,118.0,Systolic Blood Pressure,mmHg,48,Female,EMIRATI,0
2,171439,U07.1,U07,COVID 19 virus identified,Normal,Not Detected,NaN,COVID-19 PCR,66.0,Diastolic Blood Pressure,mmHg,48,Female,EMIRATI,0
3,171439,U07.1,U07,COVID 19 virus identified,Normal,Not Detected,NaN,COVID-19 PCR,56.0,Weight,kg,48,Female,EMIRATI,0
4,171439,U07.1,U07,COVID 19 virus identified,Normal,Not Detected,NaN,COVID-19 PCR,142.0,Height,cm,48,Female,EMIRATI,0


In [9]:
non_diabetic_df = non_diabetic_df.groupby('Patient').last().reset_index()

In [10]:
print("The number of distinct patients in the new df are: ", non_diabetic_df['Patient'].nunique())

The number of distinct patients in the new df are:  180739


In [13]:
for col in non_diabetic_df.columns:
    print(col)

Patient
Diagnosis_Code
DiagnosisGroup
Diagnosis_Description
ResultInterpretation
ResultValue
ResultValueUnits
TestItemCode_Description
ObservationValue
ObservationCode_Description
ObservationCode_ObservationValueUnits_Code
Age
Gender
Citizen
Target


In [14]:
non_diabetic_df.rename(columns= {'Citizenship_Description': 'Citizen', 'Gender_Name': 'Gender'}, inplace = True)

In [17]:
#distinct count of the diagnoisisgroup count
non_diabetic_df['DiagnosisGroup'].nunique()

1240

In [18]:
sorted_diagnosis_group = non_diabetic_df.groupby('DiagnosisGroup')['Patient'].count().sort_values(ascending = False).reset_index()

In [19]:
sorted_diagnosis_group

,DiagnosisGroup,Patient
0,J06,10749
1,J00,8464
2,Z02,4577
3,Z00,4377
4,R10,3910
...,...,...
1235,L86,1
1236,001,1
1237,L62,1
1238,L54,1


In [22]:
#Plotting the diagnosis group with go

#Only the top 10 diagnosis groups are plotted

fig = go.Figure(go.Bar(x = sorted_diagnosis_group['DiagnosisGroup'], y = sorted_diagnosis_group['Patient'][:20], marker_color = 'lightsalmon'))
fig.update_layout(title = 'Diagnosis Group Count', xaxis_title = 'Diagnosis Group', yaxis_title = 'Count', template = 'plotly_dark')
fig.show()

In [23]:
#Lets check the highest diagnosis group

fig = px.pie(sorted_diagnosis_group, values = 'Patient', names = 'DiagnosisGroup', title = 'Diagnosis Group Distribution')
fig.update_traces(textposition = 'inside', textinfo = 'percent+label')
fig.show()


In [15]:
#How many distinct values does Diagnosis_Code have if we group by DiagnosisGroup
non_diabetic_df.groupby('DiagnosisGroup')['Diagnosis_Code'].nunique()

DiagnosisGroup
0.0     1
001     1
1       1
10      1
15      1
       ..
Z97     4
Z98    14
Z99     2
mmc     1
pro     1
Name: Diagnosis_Code, Length: 1240, dtype: int64

In [28]:
def split_col(data):
    data['Height (in cm)'] = data.apply(lambda x: x['ObservationValue'] if 'Height' in x['ObservationCode_Description'] else np.nan, axis=1)
    data['Weight (in kg)'] = data.apply(lambda x: x['ObservationValue'] if 'Weight' in x['ObservationCode_Description'] else np.nan, axis=1)
    data['BMI in kg/m2'] = data.apply(lambda x: x['ObservationValue'] if 'BMI' in x['ObservationCode_Description'] else np.nan, axis=1)
    data['Body Temperature (in DegC)'] = data.apply(lambda x: x['ObservationValue'] if 'Body Temperature' in x['ObservationCode_Description'] else np.nan, axis=1)
    data['Diastolic Blood Pressure in mmHg'] = data.apply(lambda x: x['ObservationValue'] if 'Diastolic Blood Pressure' in x['ObservationCode_Description'] else np.nan, axis=1)
    data['Systolic Blood Pressure in mmHg'] = data.apply(lambda x: x['ObservationValue'] if 'Systolic Blood Pressure' in x['ObservationCode_Description'] else np.nan, axis=1)
    data['Heart Rate in bpm'] = data.apply(lambda x: x['ObservationValue'] if 'Heart Rate' in x['ObservationCode_Description'] else np.nan, axis=1)
    data['Respiratory Rate in bpm'] = data.apply(lambda x: x['ObservationValue'] if 'Respiratory Rate' in x['ObservationCode_Description'] else np.nan, axis=1)   

    return data

In [29]:
split_col(non_diabetic_df)

,Patient,Diagnosis_Code,DiagnosisGroup,Diagnosis_Description,ResultInterpretation,ResultValue,ResultValueUnits,TestItemCode_Description,ObservationValue,ObservationCode_Description,...,Citizen,Target,Height (in cm),Weight (in kg),BMI in kg/m2,Body Temperature (in DegC),Diastolic Blood Pressure in mmHg,Systolic Blood Pressure in mmHg,Heart Rate in bpm,Respiratory Rate in bpm
0,59268,Z83.49,Z83,"Family history of other endocrine, nutritional...",Normal,1342 HRS,mg/dl,READING TIME,96,Systolic Blood Pressure,...,UNITED ARAB EMIRATES,0,NaN,NaN,NaN,NaN,NaN,96,NaN,NaN
1,60855,J30.9,J30,"Allergic rhinitis, unspecified",Normal,36.6,SEC,APTT,13.7,Weight,...,PAKISTANI,0,NaN,13.7,NaN,NaN,NaN,NaN,NaN,NaN
2,60992,E56.9,E56,"Vitamin deficiency, unspecified",Normal,0.92,mIU/ml\X0D\\X0A\,BETA H C G LEVELS,63.6,Weight,...,Unknown,0,NaN,63.6,NaN,NaN,NaN,NaN,NaN,NaN
3,61708,K04.02,K04,Irreversible pulpitis,I,Positive,None,INFLUENZA A ANTIGEN,32,Weight,...,Emirati,0,NaN,32,NaN,NaN,NaN,NaN,NaN,NaN
4,63545,H10.013,H10,"Acute follicular conjunctivitis, bilateral",Normal,30.9,Seconds,"Partial Thromboplastin Time, test",1,Diastolic Blood Pressure,...,Palestinian,0,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180734,3230511,Z23,Z23,Encounter for immunization,None,MicroBiology Report,Null,AEROBIC CULTURE,81,Height,...,OMANI,0,81,NaN,NaN,NaN,NaN,NaN,NaN,NaN
180735,3230702,K02.52,K02,Dental caries on pit and fissure surface penet...,I,TURBID,/HPF,CLARITY,80.000,Diastolic Blood Pressure,...,INDIAN,0,NaN,NaN,NaN,NaN,80.000,NaN,NaN,NaN
180736,3230708,M62.838,M62,Other muscle spasm,Normal,5.28,mIU/mL,FOLLICLE STIMULATING HORMONE,66,Weight,...,JORDANIAN,0,NaN,66,NaN,NaN,NaN,NaN,NaN,NaN
180737,3230715,S90.30XA,S90,"Contusion of unspecified foot, initial encounter",Normal,Not Detected,None,COVID-19 PCR,20,Weight,...,EMIRATI,0,NaN,20,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
def null_columns(data):
    null_df = pd.DataFrame(data.isnull().sum(), columns = ["Null Values"])
    null_df["% Null Values"] = round((null_df["Null Values"]/len(data))*100, 2)
    return null_df[null_df["Null Values"] > 0]

In [32]:
null_columns(non_diabetic_df).sort_values(by = "Null Values", ascending = False)

,Null Values,% Null Values
Systolic Blood Pressure in mmHg,171691,94.99
Respiratory Rate in bpm,170811,94.51
Heart Rate in bpm,170521,94.35
Body Temperature (in DegC),167363,92.60
BMI in kg/m2,160109,88.59
Height (in cm),144313,79.85
Diastolic Blood Pressure in mmHg,141520,78.30
Weight (in kg),138853,76.83
ResultValueUnits,92234,51.03
ResultInterpretation,18025,9.97


In [64]:
#loading entire dataset
work_df = pd.read_csv('NonDiabetic.csv')

In [65]:
work_df.shape

(5957566, 14)

In [66]:
work_df.head()

,Patient,Diagnosis_Code,DiagnosisGroup,Diagnosis_Description,ResultInterpretation,ResultValue,ResultValueUnits,TestItemCode_Description,ObservationValue,ObservationCode_Description,ObservationCode_ObservationValueUnits_Code,Age,Gender_Name,Citizenship_Description
0,171439,U07.1,U07,COVID 19 virus identified,Normal,Not Detected,NaN,COVID-19 PCR,20.0,Respiratory Rate,bpm,48,Female,EMIRATI
1,171439,U07.1,U07,COVID 19 virus identified,Normal,Not Detected,NaN,COVID-19 PCR,118.0,Systolic Blood Pressure,mmHg,48,Female,EMIRATI
2,171439,U07.1,U07,COVID 19 virus identified,Normal,Not Detected,NaN,COVID-19 PCR,66.0,Diastolic Blood Pressure,mmHg,48,Female,EMIRATI
3,171439,U07.1,U07,COVID 19 virus identified,Normal,Not Detected,NaN,COVID-19 PCR,56.0,Weight,kg,48,Female,EMIRATI
4,171439,U07.1,U07,COVID 19 virus identified,Normal,Not Detected,NaN,COVID-19 PCR,142.0,Height,cm,48,Female,EMIRATI


In [67]:
for col in work_df.columns:
    print(col)

Patient
Diagnosis_Code
DiagnosisGroup
Diagnosis_Description
ResultInterpretation
ResultValue
ResultValueUnits
TestItemCode_Description
ObservationValue
ObservationCode_Description
ObservationCode_ObservationValueUnits_Code
Age
Gender_Name
Citizenship_Description


In [68]:
# work_df.drop(['TestItemCode_IsNumeric', 'TestItemCode_Code', 'PerformedAt_SDACodingStandard', 'PerformedAt_Description', 'PerformedAt_Code', 'TestItemStatus', 'TagFacility.1', 'ResultTime', 'ResultRawValue', 'ResultNormalRange',
#               'Patient.1', 'HSAAOrder', 'ObservationValueType', 'ObservationTime', 'LastUpdated.1', 'LabResult', 'Encounter.1', 'DiagnosisPriority', 'DiagnosisTime', 'Encounter', 'EncounterNumber',
#               'EnteredAt','EnteredOn', 'IsPrimary', 'LastUpdated', 'TagFacility', 'Diagnosis_Code', 'DiagnosisType_Code', 'Diagnosis_Description', 'Diagnosis_SDACodingStandard', 'DiagnosisType_Description',
#               'Status_Code', 'Status_Description', 'ID.1', 'AnalysisTime', 'ID'], axis = 1, inplace = True)

In [69]:
# work_df.drop(['ObservationCode_SDACodingStandard', 'ObservationCode_ObservationValueUnits_Description', 'ObservationCode_ObservationValueUnits_Code', 'ObservationCode_Code', 'TagFacility.2', 'Tag',
#               'Patient.2', 'ObservationValueType.1', 'ObservationTime.1', 'ObservationRawValue', 'LastUpdated.2', 'EncounterNumber.1', 'Encounter.2', 'ID.2'], axis=1, inplace=True)

In [70]:
# work_df.drop(['Expr16', 'EnteredOnChar', 'Expr15', 'Expr14', 'Expr13', 'Expr12', 'Expr11', 'Expr10', 'Expr9', 'Expr8', 'Expr7', 'Expr6', 'Expr4', 'Expr3', 'Expr2', 'Expr1'], axis = 1, inplace = True)

In [71]:
def extract_values(data, work_df):
    non_diabetic_df = data.copy()

    for i in non_diabetic_df['Patient'].unique():
        patient_data = work_df[work_df['Patient'] == i]
        
        for _, row in patient_data.iterrows():
            desc = row['ObservationCode_Description']
            value = row['ObservationValue']
            
            if 'Height' in desc:
                non_diabetic_df.loc[non_diabetic_df['Patient'] == i, 'Height (in cm)'] = value
            elif 'Weight' in desc:
                non_diabetic_df.loc[non_diabetic_df['Patient'] == i, 'Weight (in kg)'] = value
            elif 'BMI' in desc:
                non_diabetic_df.loc[non_diabetic_df['Patient'] == i, 'BMI in kg/m2'] = value
            elif 'Body Temperature' in desc:
                non_diabetic_df.loc[non_diabetic_df['Patient'] == i, 'Body Temperature (in DegC)'] = value
            elif 'Diastolic Blood Pressure' in desc:
                non_diabetic_df.loc[non_diabetic_df['Patient'] == i, 'Diastolic Blood Pressure in mmHg'] = value
            elif 'Systolic Blood Pressure' in desc:
                non_diabetic_df.loc[non_diabetic_df['Patient'] == i, 'Systolic Blood Pressure in mmHg'] = value
            elif 'Heart Rate' in desc:
                non_diabetic_df.loc[non_diabetic_df['Patient'] == i, 'Heart Rate in bpm'] = value
            elif 'Respiratory Rate' in desc:
                non_diabetic_df.loc[non_diabetic_df['Patient'] == i, 'Respiratory Rate in bpm'] = value

    return non_diabetic_df


⚠️This cell takes too long to run, dont run it when running the entire notebook.

In [72]:
#show the progress bar
import tqdm
tqdm.tqdm.pandas()
tqdm.tqdm.pandas(desc="Progress")
non_diabetic_df = extract_values(non_diabetic_df, work_df)

-- Comments : 19.86667 Hours to run this cell --

In [73]:
non_diabetic_df.head(20)

,Patient,Diagnosis_Code,DiagnosisGroup,Diagnosis_Description,ResultInterpretation,ResultValue,ResultValueUnits,TestItemCode_Description,ObservationValue,ObservationCode_Description,...,Citizen,Target,Height (in cm),Weight (in kg),BMI in kg/m2,Body Temperature (in DegC),Diastolic Blood Pressure in mmHg,Systolic Blood Pressure in mmHg,Heart Rate in bpm,Respiratory Rate in bpm
0,59268,Z83.49,Z83,"Family history of other endocrine, nutritional...",Normal,1342 HRS,mg/dl,READING TIME,96,Systolic Blood Pressure,...,UNITED ARAB EMIRATES,0,154,72,NaN,37.2,54,96,77,18
1,60855,J30.9,J30,"Allergic rhinitis, unspecified",Normal,36.6,SEC,APTT,13.7,Weight,...,PAKISTANI,0,NaN,13.7,NaN,38.5,NaN,NaN,130,28
2,60992,E56.9,E56,"Vitamin deficiency, unspecified",Normal,0.92,mIU/ml\X0D\\X0A\,BETA H C G LEVELS,63.6,Weight,...,Unknown,0,155,63.6,NaN,NaN,70,110,78,20
3,61708,K04.02,K04,Irreversible pulpitis,I,Positive,None,INFLUENZA A ANTIGEN,32,Weight,...,Emirati,0,NaN,32,NaN,38.5,NaN,NaN,126,26
4,63545,H10.013,H10,"Acute follicular conjunctivitis, bilateral",Normal,30.9,Seconds,"Partial Thromboplastin Time, test",1,Diastolic Blood Pressure,...,Palestinian,0,111,17.75,14.41,NaN,1,1,NaN,NaN
5,64977,R00.2,R00,Palpitations,Normal,19.49,pmol/L,FREE T4,18.9,Weight,...,Palestinian,0,109,18.9,NaN,37,NaN,NaN,102,20
6,65394,K80.20,K80,Calculus of gallbladder without cholecystitis ...,Normal,24.4,U/L,SGPT / ALT,157,Height,...,EMIRATI,0,157,73.6,30,36.5,79,111,74,19
7,66194,Z02.79,Z02,Encounter for issue of other medical certificate,Normal,.30,None,Hgb F %,29,BMI (Body Mass Index),...,EMIRATI,0,181,94,29,36.5,61,110,77,18
8,67249,R19.7,R19,"Diarrhea, unspecified",Normal,1.80,mIU/L,TSH,92,Heart Rate,...,EMIRATI,0,160,58,23,36.3,72,102,92,18
9,68154,R22.0,R22,"Localized swelling, mass and lump, head",High,5.6,mg/L,C Reactive Protein,17.0,Weight,...,SYRIAN,0,NaN,17.0,NaN,37.0,NaN,NaN,98.0,NaN


In [43]:
null_columns(non_diabetic_df).sort_values(by = "Null Values", ascending = False)

,Null Values,% Null Values
BMI in kg/m2,163311,90.36
Height (in cm),159362,88.17
Weight (in kg),157873,87.35
Respiratory Rate in bpm,157677,87.24
Body Temperature (in DegC),156345,86.50
Heart Rate in bpm,155247,85.90
Diastolic Blood Pressure in mmHg,154551,85.51
Systolic Blood Pressure in mmHg,154489,85.48
ResultValueUnits,92234,51.03
ResultInterpretation,18025,9.97


In [44]:
#Pull the distinct values from the column TestItemCode_Description
non_diabetic_df['TestItemCode_Description'].unique()

array(['READING TIME', 'APTT', 'BETA H C G LEVELS', ...,
       'CULTURE \\T\\ SENSITIVITY, SEMINAL FLUID', 'BETA HCG (TOTAL)',
       'UA Ammon Urates'], dtype=object)

In [48]:
for i in non_diabetic_df['TestItemCode_Description'].unique():
    print(i)

# #export the output to a txt file
# with open('TestItemCode_Description.txt', 'w') as f:
#     for item in non_diabetic_df['TestItemCode_Description'].unique():
#         f.write("%s\n" % item)


READING TIME
APTT
BETA H C G LEVELS
INFLUENZA A ANTIGEN
Partial Thromboplastin Time, test
FREE T4
SGPT / ALT
Hgb F %
TSH
C Reactive Protein
Result
EPITHELIAL CELLS.
Osmolality
KETONES.
PROCALCITONIN
eGFR
REMARK
COVID-19 PCR
CA 125 , Quantitative, Serum -Atellica
Infectious agent antigen detection by immunoassay with direct optical observation; Influenza
Vitamin D 25 OH
Influenza B PCR
HbA1c
IG
PROLACTIN
THYROID STIMULATING HORMONE
COLOUR
Test Performed by:
CRP ; C-Reactive Protein, Quantitative
STOOL H PYLORI
Hgb A1c
UA Ketones
IgE
HVS Trichomonas
LH
Final
UA Protein
Hep B Vir DNA Quan
Glucose Random (RBS)
Ur Microalbumin
Influenza A PCR
Valpro Acid Lvl
AGAP
Pro BNP
CRP
UA Glucose
Hep C Ab Interp
VITAMIN D3 (25 - OH)
Ferritin Lvl
Zinc Lvl
Vitamin B12 Lvl
T4 Free
Influenza A Ab
UA pH
AMA-M2-IgG Interp
Beta HCG (BHCG), Quantitative, Serum
UA Squam Epi
Ferritin*
UA RBC
FIT
DCon
Prothrombin INR
Number of Units/mL
SC I
Glucose Random (RBS), Fluoride Plasma
MCV
Antihuman Globulin Test ( Coom

In [74]:
null_columns(non_diabetic_df).sort_values(by="Null Values", ascending=False)

,Null Values,% Null Values
BMI in kg/m2,98388,54.44
ResultValueUnits,92234,51.03
Height (in cm),51642,28.57
Diastolic Blood Pressure in mmHg,37637,20.82
Systolic Blood Pressure in mmHg,37275,20.62
Weight (in kg),35206,19.48
Respiratory Rate in bpm,33081,18.30
Heart Rate in bpm,19265,10.66
Body Temperature (in DegC),18671,10.33
ResultInterpretation,18025,9.97


In [75]:
non_diabetic_df.shape

(180739, 23)

In [76]:
non_diabetic_df.to_csv('FinalNonDiabetic.csv', index = False)

In [77]:
for col in non_diabetic_df.columns:
    print(col)

Patient
Diagnosis_Code
DiagnosisGroup
Diagnosis_Description
ResultInterpretation
ResultValue
ResultValueUnits
TestItemCode_Description
ObservationValue
ObservationCode_Description
ObservationCode_ObservationValueUnits_Code
Age
Gender
Citizen
Target
Height (in cm)
Weight (in kg)
BMI in kg/m2
Body Temperature (in DegC)
Diastolic Blood Pressure in mmHg
Systolic Blood Pressure in mmHg
Heart Rate in bpm
Respiratory Rate in bpm


In [79]:
non_diabetic_df.drop(['ObservationValue','ObservationCode_Description', 'ObservationCode_ObservationValueUnits_Code', 'DiagnosisGroup'], axis=1, inplace=True)

In [80]:
non_diabetic_df.shape

(180739, 19)

In [81]:
non_diabetic_df.to_csv('FinalNonDiabetic.csv', index = False)

In [83]:
testitem = pd.read_csv('TestItemCode_Description.csv')

In [84]:
testitem.head()

,Required,TestItemCodeDescription,Standardized Name
0,N,%Active,NaN
1,N,%FREE PSA,NaN
2,N,%NonMot 2hr,NaN
3,N,%Slugg 2hr,NaN
4,N,* ABO GROUPING,NaN


In [86]:
#Distinct values in the column Standardized Name
testitem['Standardized Name'].unique()    

array([nan, 'ALB', 'ALP', 'ALT', 'AMY', 'CAL', 'CRP', 'CRTN', 'DBIL',
       'ESR', 'FBS', 'FSH', 'FT3', 'FT4', 'PPBS', 'HDL', 'HbA1c', 'LDL',
       'MGSM', 'POTSM', 'RBS', 'TBIL', 'TCHOL', 'TRIG', 'TROPI', 'UREA',
       'URIC', 'VITD', 'CORT', 'AST', 'IBIL', 'GRBS', 'KTN', 'CK',
       'CRPhs', 'RUC', 'DHEAS', 'FIS', 'TORPIhs', 'INS', 'INSPP', 'FINS',
       'ZINC', 'TSH', 'PRTN', 'TROPT', 'TROPIhs', 'TROPThs', 'UGC', 'UKA',
       'VLDL'], dtype=object)

In [88]:
#Now we will group the TestItemCodeDescription, according to the Standardized Name
testitem.groupby('Standardized Name')['TestItemCodeDescription'].nunique()

Standardized Name
ALB         8
ALP        12
ALT        17
AMY        11
AST        16
CAL        16
CK          9
CORT        9
CRP        21
CRPhs       2
CRTN       15
DBIL       11
DHEAS       4
ESR        13
FBS        23
FINS        8
FIS         1
FSH         6
FT3        14
FT4        12
GRBS        7
HDL        17
HbA1c      20
IBIL        3
INS         5
INSPP       6
KTN         8
LDL        21
MGSM        7
POTSM      13
PPBS       41
PRTN        1
RBS        26
RUC         3
TBIL       18
TCHOL      22
TORPIhs     3
TRIG       15
TROPI      11
TROPIhs     1
TROPT       5
TROPThs     4
TSH        13
UGC         4
UKA         3
UREA       14
URIC        1
VITD       23
VLDL        8
ZINC        8
Name: TestItemCodeDescription, dtype: int64

In [90]:
# we will now list the tests only that are required with Y
testitem = testitem[testitem['Required'] == 'Y']
testitem.head()


,Required,TestItemCodeDescription,Standardized Name
7,Y,* ALBUMIN,ALB
8,Y,* Albumin,ALB
9,Y,* ALKALINE PHOSPHATASE,ALP
10,Y,* ALT (SGPT),ALT
11,Y,* AMYLASE,AMY


In [91]:
#lets check the distinct values in the column Standardized Name
testitem['Standardized Name'].unique()


array(['ALB', 'ALP', 'ALT', 'AMY', 'CAL', 'CRP', 'CRTN', 'DBIL', 'ESR',
       'FBS', 'FSH', 'FT3', 'FT4', 'PPBS', 'HDL', 'HbA1c', 'LDL', 'MGSM',
       'POTSM', 'RBS', 'TBIL', 'TCHOL', 'TRIG', 'TROPI', 'UREA', 'URIC',
       'VITD', 'CORT', 'AST', 'IBIL', 'GRBS', 'KTN', 'CK', 'CRPhs', 'RUC',
       'DHEAS', 'FIS', 'TORPIhs', 'INS', 'INSPP', 'FINS', 'ZINC', 'TSH',
       'PRTN', 'TROPT', 'TROPIhs', 'TROPThs', 'UGC', 'UKA', 'VLDL'],
      dtype=object)

In [92]:
#replace the values in the column TestItemCodeDescription with the Standardized Name
non_diabetic_df['TestItemCode_Description'] = non_diabetic_df['TestItemCode_Description'].replace(testitem.set_index('TestItemCodeDescription')['Standardized Name'].to_dict())


In [93]:
non_diabetic_df.head()

,Patient,Diagnosis_Code,Diagnosis_Description,ResultInterpretation,ResultValue,ResultValueUnits,TestItemCode_Description,Age,Gender,Citizen,Target,Height (in cm),Weight (in kg),BMI in kg/m2,Body Temperature (in DegC),Diastolic Blood Pressure in mmHg,Systolic Blood Pressure in mmHg,Heart Rate in bpm,Respiratory Rate in bpm
0,59268,Z83.49,"Family history of other endocrine, nutritional...",Normal,1342 HRS,mg/dl,READING TIME,41,Female,UNITED ARAB EMIRATES,0,154,72,NaN,37.2,54,96,77,18
1,60855,J30.9,"Allergic rhinitis, unspecified",Normal,36.6,SEC,APTT,4,Male,PAKISTANI,0,NaN,13.7,NaN,38.5,NaN,NaN,130,28
2,60992,E56.9,"Vitamin deficiency, unspecified",Normal,0.92,mIU/ml\X0D\\X0A\,BETA H C G LEVELS,41,Female,Unknown,0,155,63.6,NaN,NaN,70,110,78,20
3,61708,K04.02,Irreversible pulpitis,I,Positive,None,INFLUENZA A ANTIGEN,5,Male,Emirati,0,NaN,32,NaN,38.5,NaN,NaN,126,26
4,63545,H10.013,"Acute follicular conjunctivitis, bilateral",Normal,30.9,Seconds,"Partial Thromboplastin Time, test",4,Female,Palestinian,0,111,17.75,14.41,NaN,1,1,NaN,NaN


In [94]:
#how manmy rows have null values in the column TestItemCode_Description
non_diabetic_df['TestItemCode_Description'].isnull().sum()

0

In [97]:
#Replace the TestItemCode_Description in the non_diabetic_df with the values in the Standardized Name column in the testitem

non_diabetic_df['TestItemCode_Description'] = non_diabetic_df['TestItemCode_Description'].map(testitem.set_index('TestItemCodeDescription')['Standardized Name'])



In [98]:
non_diabetic_df.head()

,Patient,Diagnosis_Code,Diagnosis_Description,ResultInterpretation,ResultValue,ResultValueUnits,TestItemCode_Description,Age,Gender,Citizen,Target,Height (in cm),Weight (in kg),BMI in kg/m2,Body Temperature (in DegC),Diastolic Blood Pressure in mmHg,Systolic Blood Pressure in mmHg,Heart Rate in bpm,Respiratory Rate in bpm
0,59268,Z83.49,"Family history of other endocrine, nutritional...",Normal,1342 HRS,mg/dl,NaN,41,Female,UNITED ARAB EMIRATES,0,154,72,NaN,37.2,54,96,77,18
1,60855,J30.9,"Allergic rhinitis, unspecified",Normal,36.6,SEC,NaN,4,Male,PAKISTANI,0,NaN,13.7,NaN,38.5,NaN,NaN,130,28
2,60992,E56.9,"Vitamin deficiency, unspecified",Normal,0.92,mIU/ml\X0D\\X0A\,NaN,41,Female,Unknown,0,155,63.6,NaN,NaN,70,110,78,20
3,61708,K04.02,Irreversible pulpitis,I,Positive,None,NaN,5,Male,Emirati,0,NaN,32,NaN,38.5,NaN,NaN,126,26
4,63545,H10.013,"Acute follicular conjunctivitis, bilateral",Normal,30.9,Seconds,NaN,4,Female,Palestinian,0,111,17.75,14.41,NaN,1,1,NaN,NaN


In [99]:
non_diabetic_df['TestItemCode_Description'].isnull().sum()

154892

In [101]:
non_diabetic_df.shape

(180739, 19)

In [102]:
#count of each value in the column TestItemCode_Description
non_diabetic_df['TestItemCode_Description'].value_counts()

TestItemCode_Description
HbA1c    11772
CRP       6631
TSH       2840
ALT        944
ESR        664
GRBS       650
UREA       451
AST        380
PPBS       376
LDL        376
ALP        285
HDL        263
FSH        157
DHEAS       31
VLDL        27
Name: count, dtype: int64

In [104]:
work_df.shape

(5957566, 14)

In [105]:
#Replace the TestItemCode_Description in the non_diabetic_df with the values in the Standardized Name column in the testitem

work_df['TestItemCode_Description'] = work_df['TestItemCode_Description'].map(testitem.set_index('TestItemCodeDescription')['Standardized Name'])

In [106]:
work_df['TestItemCode_Description'].value_counts()

TestItemCode_Description
HbA1c      155689
CRP        103157
RBS         68604
VITD        60796
TSH         57172
TBIL        51126
UGC         45467
UKA         44872
CRTN        44040
ALT         41290
FT4         38285
POTSM       34479
UREA        31835
AST         30451
TRIG        28653
ALB         28541
LDL         27649
TCHOL       27527
ALP         25810
HDL         24964
KTN         21846
PRTN        21187
CAL         13417
GRBS        10978
FBS         10521
ESR          9011
DBIL         6303
FT3          6120
PPBS         5958
MGSM         5522
FSH          5334
VLDL         5047
AMY          4946
FINS         2305
INSPP        1871
TROPThs      1804
TROPI        1753
RUC          1564
ZINC         1352
DHEAS        1259
IBIL         1155
URIC          827
CK            651
CORT          582
TROPT         537
INS           326
TROPIhs       177
TORPIhs        98
CRPhs          89
FIS            57
Name: count, dtype: int64

In [107]:
#Now we will convert these TestItemCode_Description values into columns
work_df = pd.get_dummies(work_df, columns = ['TestItemCode_Description'], prefix = ['TestItemCode_Description'])


In [108]:
work_df.head()

,Patient,Diagnosis_Code,DiagnosisGroup,Diagnosis_Description,ResultInterpretation,ResultValue,ResultValueUnits,ObservationValue,ObservationCode_Description,ObservationCode_ObservationValueUnits_Code,...,TestItemCode_Description_TROPT,TestItemCode_Description_TROPThs,TestItemCode_Description_TSH,TestItemCode_Description_UGC,TestItemCode_Description_UKA,TestItemCode_Description_UREA,TestItemCode_Description_URIC,TestItemCode_Description_VITD,TestItemCode_Description_VLDL,TestItemCode_Description_ZINC
0,171439,U07.1,U07,COVID 19 virus identified,Normal,Not Detected,NaN,20.0,Respiratory Rate,bpm,...,False,False,False,False,False,False,False,False,False,False
1,171439,U07.1,U07,COVID 19 virus identified,Normal,Not Detected,NaN,118.0,Systolic Blood Pressure,mmHg,...,False,False,False,False,False,False,False,False,False,False
2,171439,U07.1,U07,COVID 19 virus identified,Normal,Not Detected,NaN,66.0,Diastolic Blood Pressure,mmHg,...,False,False,False,False,False,False,False,False,False,False
3,171439,U07.1,U07,COVID 19 virus identified,Normal,Not Detected,NaN,56.0,Weight,kg,...,False,False,False,False,False,False,False,False,False,False
4,171439,U07.1,U07,COVID 19 virus identified,Normal,Not Detected,NaN,142.0,Height,cm,...,False,False,False,False,False,False,False,False,False,False


In [109]:
work_df['TestItemCode_Description_UGC'].value_counts()

TestItemCode_Description_UGC
False    5912099
True       45467
Name: count, dtype: int64

In [110]:
non_diabetic_df.shape

(180739, 19)

In [111]:
non_diabetic_df.head()

,Patient,Diagnosis_Code,Diagnosis_Description,ResultInterpretation,ResultValue,ResultValueUnits,TestItemCode_Description,Age,Gender,Citizen,Target,Height (in cm),Weight (in kg),BMI in kg/m2,Body Temperature (in DegC),Diastolic Blood Pressure in mmHg,Systolic Blood Pressure in mmHg,Heart Rate in bpm,Respiratory Rate in bpm
0,59268,Z83.49,"Family history of other endocrine, nutritional...",Normal,1342 HRS,mg/dl,NaN,41,Female,UNITED ARAB EMIRATES,0,154,72,NaN,37.2,54,96,77,18
1,60855,J30.9,"Allergic rhinitis, unspecified",Normal,36.6,SEC,NaN,4,Male,PAKISTANI,0,NaN,13.7,NaN,38.5,NaN,NaN,130,28
2,60992,E56.9,"Vitamin deficiency, unspecified",Normal,0.92,mIU/ml\X0D\\X0A\,NaN,41,Female,Unknown,0,155,63.6,NaN,NaN,70,110,78,20
3,61708,K04.02,Irreversible pulpitis,I,Positive,None,NaN,5,Male,Emirati,0,NaN,32,NaN,38.5,NaN,NaN,126,26
4,63545,H10.013,"Acute follicular conjunctivitis, bilateral",Normal,30.9,Seconds,NaN,4,Female,Palestinian,0,111,17.75,14.41,NaN,1,1,NaN,NaN


In [115]:
non_diabetic_df.head()

,Patient,Diagnosis_Code,Diagnosis_Description,ResultInterpretation,ResultValue,ResultValueUnits,Age,Gender,Citizen,Target,...,TestItemCode_Description_ESR,TestItemCode_Description_FSH,TestItemCode_Description_GRBS,TestItemCode_Description_HDL,TestItemCode_Description_HbA1c,TestItemCode_Description_LDL,TestItemCode_Description_PPBS,TestItemCode_Description_TSH,TestItemCode_Description_UREA,TestItemCode_Description_VLDL
0,59268,Z83.49,"Family history of other endocrine, nutritional...",Normal,1342 HRS,mg/dl,41,Female,UNITED ARAB EMIRATES,0,...,False,False,False,False,False,False,False,False,False,False
1,60855,J30.9,"Allergic rhinitis, unspecified",Normal,36.6,SEC,4,Male,PAKISTANI,0,...,False,False,False,False,False,False,False,False,False,False
2,60992,E56.9,"Vitamin deficiency, unspecified",Normal,0.92,mIU/ml\X0D\\X0A\,41,Female,Unknown,0,...,False,False,False,False,False,False,False,False,False,False
3,61708,K04.02,Irreversible pulpitis,I,Positive,None,5,Male,Emirati,0,...,False,False,False,False,False,False,False,False,False,False
4,63545,H10.013,"Acute follicular conjunctivitis, bilateral",Normal,30.9,Seconds,4,Female,Palestinian,0,...,False,False,False,False,False,False,False,False,False,False


In [117]:
non_diabetic_df['TestItemCode_Description_HbA1c'].value_counts()

TestItemCode_Description_HbA1c
False    168967
True      11772
Name: count, dtype: int64

In [118]:
non_diabetic_df.dtypes

Patient                              int64
Diagnosis_Code                      object
Diagnosis_Description               object
ResultInterpretation                object
ResultValue                         object
ResultValueUnits                    object
Age                                  int64
Gender                              object
Citizen                             object
Target                               int64
Height (in cm)                      object
Weight (in kg)                      object
BMI in kg/m2                        object
Body Temperature (in DegC)          object
Diastolic Blood Pressure in mmHg    object
Systolic Blood Pressure in mmHg     object
Heart Rate in bpm                   object
Respiratory Rate in bpm             object
TestItemCode_Description_ALP          bool
TestItemCode_Description_ALT          bool
TestItemCode_Description_AST          bool
TestItemCode_Description_CRP          bool
TestItemCode_Description_DHEAS        bool
TestItemCod

In [119]:
non_diabetic_df.to_csv('TestCheck.csv', index = False)

In [120]:
entire_df = pd.read_csv('NonDiabetic.csv')

In [121]:
#replace the values in the column TestItemCodeDescription with the Standardized Name, and if the value is not present, then replace it with the TestItemCodeDescription
entire_df['TestItemCode_Description'] = entire_df['TestItemCode_Description'].replace(testitem.set_index('TestItemCodeDescription')['Standardized Name'].to_dict())


In [122]:
entire_df.head()

,Patient,Diagnosis_Code,DiagnosisGroup,Diagnosis_Description,ResultInterpretation,ResultValue,ResultValueUnits,TestItemCode_Description,ObservationValue,ObservationCode_Description,ObservationCode_ObservationValueUnits_Code,Age,Gender_Name,Citizenship_Description
0,171439,U07.1,U07,COVID 19 virus identified,Normal,Not Detected,NaN,COVID-19 PCR,20.0,Respiratory Rate,bpm,48,Female,EMIRATI
1,171439,U07.1,U07,COVID 19 virus identified,Normal,Not Detected,NaN,COVID-19 PCR,118.0,Systolic Blood Pressure,mmHg,48,Female,EMIRATI
2,171439,U07.1,U07,COVID 19 virus identified,Normal,Not Detected,NaN,COVID-19 PCR,66.0,Diastolic Blood Pressure,mmHg,48,Female,EMIRATI
3,171439,U07.1,U07,COVID 19 virus identified,Normal,Not Detected,NaN,COVID-19 PCR,56.0,Weight,kg,48,Female,EMIRATI
4,171439,U07.1,U07,COVID 19 virus identified,Normal,Not Detected,NaN,COVID-19 PCR,142.0,Height,cm,48,Female,EMIRATI


In [123]:
entire_df['TestItemCode_Description'].value_counts()

TestItemCode_Description
COVID-19 PCR              606178
HbA1c                     155689
CRP                       103157
MCHC                       80890
MCV                        80666
                           ...  
D- Dimer                       1
UA Waxy Cast                   1
NRBC Man                       1
HbA1C(GLYCOSYLATED Hb)         1
D Dimer, Plasma                1
Name: count, Length: 2864, dtype: int64

In [124]:
non_diabetic_df.head()

,Patient,Diagnosis_Code,Diagnosis_Description,ResultInterpretation,ResultValue,ResultValueUnits,Age,Gender,Citizen,Target,...,TestItemCode_Description_ESR,TestItemCode_Description_FSH,TestItemCode_Description_GRBS,TestItemCode_Description_HDL,TestItemCode_Description_HbA1c,TestItemCode_Description_LDL,TestItemCode_Description_PPBS,TestItemCode_Description_TSH,TestItemCode_Description_UREA,TestItemCode_Description_VLDL
0,59268,Z83.49,"Family history of other endocrine, nutritional...",Normal,1342 HRS,mg/dl,41,Female,UNITED ARAB EMIRATES,0,...,False,False,False,False,False,False,False,False,False,False
1,60855,J30.9,"Allergic rhinitis, unspecified",Normal,36.6,SEC,4,Male,PAKISTANI,0,...,False,False,False,False,False,False,False,False,False,False
2,60992,E56.9,"Vitamin deficiency, unspecified",Normal,0.92,mIU/ml\X0D\\X0A\,41,Female,Unknown,0,...,False,False,False,False,False,False,False,False,False,False
3,61708,K04.02,Irreversible pulpitis,I,Positive,None,5,Male,Emirati,0,...,False,False,False,False,False,False,False,False,False,False
4,63545,H10.013,"Acute follicular conjunctivitis, bilateral",Normal,30.9,Seconds,4,Female,Palestinian,0,...,False,False,False,False,False,False,False,False,False,False


In [125]:
for col in non_diabetic_df.columns:
    print(col)

Patient
Diagnosis_Code
Diagnosis_Description
ResultInterpretation
ResultValue
ResultValueUnits
Age
Gender
Citizen
Target
Height (in cm)
Weight (in kg)
BMI in kg/m2
Body Temperature (in DegC)
Diastolic Blood Pressure in mmHg
Systolic Blood Pressure in mmHg
Heart Rate in bpm
Respiratory Rate in bpm
TestItemCode_Description_ALP
TestItemCode_Description_ALT
TestItemCode_Description_AST
TestItemCode_Description_CRP
TestItemCode_Description_DHEAS
TestItemCode_Description_ESR
TestItemCode_Description_FSH
TestItemCode_Description_GRBS
TestItemCode_Description_HDL
TestItemCode_Description_HbA1c
TestItemCode_Description_LDL
TestItemCode_Description_PPBS
TestItemCode_Description_TSH
TestItemCode_Description_UREA
TestItemCode_Description_VLDL


In [151]:
def extract_testvalue(data, entire_df):
    non_diabetic_df = data.copy()

    test_names = ['ALP', 'ALT', 'AST', 'CRP', 'DHEAS', 'ESR', 'FSH', 'GRBS', 'HDL', 'HbA1c', 'LDL', 'PPBS', 'TSH', 'UREA', 'VLDL']

    for test_name in test_names:
        column_name = f'TestItemCode_Description_{test_name}'
        non_diabetic_df[column_name] = np.nan

    for i in non_diabetic_df['Patient'].unique():
        patient_data = entire_df[entire_df['Patient'] == i]

        for _, row in patient_data.iterrows():
            desc = row['TestItemCode_Description']
            value = row['ResultValue']

            for test_name in test_names:
                column_name = f'TestItemCode_Description_{test_name}'
                if test_name in desc:
                    non_diabetic_df.loc[non_diabetic_df['Patient'] == i, column_name] = value

    return non_diabetic_df


In [152]:
extract_testvalue(non_diabetic_df, entire_df)

,Patient,Diagnosis_Code,Diagnosis_Description,ResultInterpretation,ResultValue,ResultValueUnits,Age,Gender,Citizen,Target,...,TestItemCode_Description_ESR,TestItemCode_Description_FSH,TestItemCode_Description_GRBS,TestItemCode_Description_HDL,TestItemCode_Description_HbA1c,TestItemCode_Description_LDL,TestItemCode_Description_PPBS,TestItemCode_Description_TSH,TestItemCode_Description_UREA,TestItemCode_Description_VLDL
0,59268,Z83.49,"Family history of other endocrine, nutritional...",Normal,1342 HRS,mg/dl,41,Female,UNITED ARAB EMIRATES,0,...,NaN,NaN,95,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,60855,J30.9,"Allergic rhinitis, unspecified",Normal,36.6,SEC,4,Male,PAKISTANI,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,60992,E56.9,"Vitamin deficiency, unspecified",Normal,0.92,mIU/ml\X0D\\X0A\,41,Female,Unknown,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,61708,K04.02,Irreversible pulpitis,I,Positive,None,5,Male,Emirati,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,63545,H10.013,"Acute follicular conjunctivitis, bilateral",Normal,30.9,Seconds,4,Female,Palestinian,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180734,3230511,Z23,Encounter for immunization,None,MicroBiology Report,Null,1,Female,OMANI,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
180735,3230702,K02.52,Dental caries on pit and fissure surface penet...,I,TURBID,/HPF,31,Female,INDIAN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
180736,3230708,M62.838,Other muscle spasm,Normal,5.28,mIU/mL,29,Female,JORDANIAN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
180737,3230715,S90.30XA,"Contusion of unspecified foot, initial encounter",Normal,Not Detected,None,6,Male,EMIRATI,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [153]:
null_columns(non_diabetic_df).sort_values(by = "Null Values", ascending = False)

,Null Values,% Null Values
BMI in kg/m2,98388,54.44
ResultValueUnits,92234,51.03
Height (in cm),51642,28.57
Diastolic Blood Pressure in mmHg,37637,20.82
Systolic Blood Pressure in mmHg,37275,20.62
Weight (in kg),35206,19.48
Respiratory Rate in bpm,33081,18.30
Heart Rate in bpm,19265,10.66
Body Temperature (in DegC),18671,10.33
ResultInterpretation,18025,9.97
